In [ ]:
import pandas as pd
import numpy as np

# ランダムシード設定（再現性のため）
np.random.seed(42)

# データ数
n_samples = 2000  # より多くのデータで学習効果向上

# 東京主要区（23区）
wards = ['千代田区', '中央区', '港区', '新宿区', '文京区', '台東区', '墨田区', 
         '江東区', '品川区', '目黒区', '大田区', '世田谷区', '渋谷区', '中野区',
         '杉並区', '豊島区', '北区', '荒川区', '板橋区', '練馬区', '足立区', '葛飾区', '江戸川区']

# 区別の基本価格 - 非常に極端に設定して区の影響を明確に
ward_base_price = {
    # 超高級エリア - 基本価格を非常に高く設定
    '港区': 200000,   # 最高級
    '千代田区': 180000,
    '中央区': 160000,
    '渋谷区': 150000,   
    
    # 高級エリア
    '目黒区': 130000,
    '文京区': 125000,
    '新宿区': 120000,
    '品川区': 115000,
    '世田谷区': 110000,
    
    # 中価格帯エリア
    '豊島区': 90000,
    '台東区': 85000,
    '中野区': 85000,     
    '杉並区': 85000,
    '江東区': 80000,
    '大田区': 80000,
    '墨田区': 75000,
    '練馬区': 75000,
    
    # 低価格帯エリア - 기본 가격을 매우 낮게 설정
    '北区': 60000,      
    '板橋区': 55000,    
    '荒川区': 50000,    
    '江戸川区': 50000,  
    '葛飾区': 45000,    
    '足立区': 40000     # 最低価格
}

# 建物構造 - 影響力減少
structures = ['木造', 'RC造', '鉄骨造', 'SRC造']
structure_adjustments = {'木造': -5000, '鉄骨造': 0, 'RC造': 5000, 'SRC造': 10000}

# 建物タイプ - 影響力減少
property_types = ['マンション', 'アパート', 'ハイツ', 'コーポ']
type_adjustments = {'マンション': 10000, 'アパート': 0, 'ハイツ': -3000, 'コーポ': -5000}

# 区別の均等サンプリング設定
samples_per_ward = n_samples // len(wards)
remaining_samples = n_samples % len(wards)

# データ生成
data = []
ward_sample_counts = {ward: samples_per_ward for ward in wards}
# 残りサンプルを高級地域に追加割当
for ward in ['港区', '千代田区', '中央区', '渋谷区']:
    if remaining_samples > 0:
        ward_sample_counts[ward] += 1
        remaining_samples -= 1

for ward in wards:
    for _ in range(ward_sample_counts[ward]):
        # 区の基本価格から開始
        base_price = ward_base_price[ward]
        
        # 部屋サイズ - 区によって異なる分布
        if base_price >= 150000:  # 超高級地域
            room_size = np.random.normal(45, 15)  # 平均45m²、標準偏差15
            room_size = np.clip(room_size, 20, 100)
        elif base_price >= 100000:  # 高級地域
            room_size = np.random.normal(35, 12)  # 平均35m²
            room_size = np.clip(room_size, 18, 80)
        elif base_price >= 70000:  # 中間地域
            room_size = np.random.normal(30, 10)  # 平均30m²
            room_size = np.clip(room_size, 15, 70)
        else:  # 低価格地域
            room_size = np.random.normal(25, 8)  # 平均25m²
            room_size = np.clip(room_size, 15, 60)
        room_size = int(room_size)
        
        # 建物構造 - 高級地域ほどRC/SRCが多い
        if base_price >= 150000:
            structure = np.random.choice(structures, p=[0.05, 0.45, 0.2, 0.3])
        elif base_price >= 100000:
            structure = np.random.choice(structures, p=[0.1, 0.5, 0.25, 0.15])
        elif base_price >= 70000:
            structure = np.random.choice(structures, p=[0.2, 0.4, 0.3, 0.1])
        else:
            structure = np.random.choice(structures, p=[0.4, 0.3, 0.25, 0.05])
        
        # 建物タイプ - 高級地域ほどマンションが多い
        if base_price >= 150000:
            property_type = np.random.choice(property_types, p=[0.85, 0.1, 0.03, 0.02])
        elif base_price >= 100000:
            property_type = np.random.choice(property_types, p=[0.65, 0.25, 0.05, 0.05])
        elif base_price >= 70000:
            property_type = np.random.choice(property_types, p=[0.4, 0.4, 0.1, 0.1])
        else:
            property_type = np.random.choice(property_types, p=[0.2, 0.5, 0.15, 0.15])
        
        # 駅までの距離 - 高級地域ほど駅近
        if base_price >= 150000:
            station_distance = np.random.choice(range(1, 16), p=[0.2, 0.15, 0.15, 0.1, 0.1, 0.08, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01, 0.005, 0.003, 0.002])
        elif base_price >= 70000:
            station_distance = np.random.randint(1, 21)
        else:
            station_distance = np.random.choice(range(3, 21))  # 低価格地域は駅から遠い可能性が高い
        
        # 築年数 - 高級地域ほど新築
        if base_price >= 150000:
            building_age = np.random.choice(range(0, 31), p=[0.1] + [0.9/30]*30)  # 新築比率が高い
        elif base_price >= 100000:
            building_age = np.random.randint(0, 41)
        else:
            building_age = np.random.randint(5, 51)  # 低価格地域は古い建物が多い
        
        # 最終価格計算（区の基本価格が最も大きな影響）
        price = base_price
        
        # 部屋サイズの影響（区別に異なる適用）
        if base_price >= 150000:  # 高級地域はm²当たり価格がより高い
            price += room_size * 1500
        elif base_price >= 100000:
            price += room_size * 1200
        elif base_price >= 70000:
            price += room_size * 1000
        else:
            price += room_size * 800
        
        # 他の要素の影響（相対的に小さく）
        price += structure_adjustments[structure]
        price += type_adjustments[property_type]
        price -= station_distance * 1000  # 駅距離の影響
        price -= building_age * 300  # 築年数の影響
        
        # ノイズは小さく（±5%）
        price *= np.random.uniform(0.95, 1.05)
        
        # 最小値補正（区別の最小値も異なる）
        min_price = max(30000, base_price * 0.5)
        price = max(price, min_price)
        
        data.append({
            '区': ward,
            '部屋サイズ_m2': room_size,
            '建物構造': structure,
            '建物タイプ': property_type,
            '駅距離_分': station_distance,
            '築年数_年': building_age,
            '家賃_円': int(price)
        })

# データフレーム生成
df = pd.DataFrame(data)

# データシャッフリング
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# CSVファイルに保存
df.to_csv('tokyo_rent_data_v2.csv', index=False, encoding='utf-8-sig')

print("=" * 80)
print("強化されたデータ生成完了!（区の影響力極大化）")
print("=" * 80)
print(f"\n合計{len(df)}個のデータが生成されました。")
print("\n区別データ件数:")
print(df['区'].value_counts().sort_index())

print("\n" + "=" * 80)
print("区別平均家賃（区の差が明確かどうか確認）:")
print("=" * 80)
ward_stats = df.groupby('区')['家賃_円'].agg(['mean', 'min', 'max', 'std']).round(0)
ward_stats = ward_stats.sort_values('mean', ascending=False)
print(ward_stats.head(10))
print("\n最も安い区:")
print(ward_stats.tail(5))

# 区別の差異検証
print("\n" + "=" * 80)
print("極端な区の比較:")
print("=" * 80)
minato_avg = df[df['区'] == '港区']['家賃_円'].mean()
adachi_avg = df[df['区'] == '足立区']['家賃_円'].mean()
print(f"港区 平均: ¥{minato_avg:,.0f}")
print(f"足立区 平均: ¥{adachi_avg:,.0f}")
print(f"差額: ¥{minato_avg - adachi_avg:,.0f}")
print(f"倍率: {minato_avg / adachi_avg:.2f}倍")

# 同一条件での区別差異確認
print("\n" + "=" * 80)
print("同一条件での区別価格差（30m², RC造, マンション, 駅5分, 築10年）:")
print("=" * 80)
for ward in ['港区', '千代田区', '渋谷区', '新宿区', '中野区', '足立区']:
    similar_conditions = df[
        (df['区'] == ward) &
        (df['部屋サイズ_m2'].between(28, 32)) &
        (df['建物構造'] == 'RC造') &
        (df['建物タイプ'] == 'マンション') &
        (df['駅距離_分'].between(4, 6)) &
        (df['築年数_年'].between(8, 12))
    ]
    if len(similar_conditions) > 0:
        avg_price = similar_conditions['家賃_円'].mean()
        print(f"{ward}: ¥{avg_price:,.0f} ({len(similar_conditions)}件)")
    else:
        # 近似条件に拡大
        similar_conditions = df[
            (df['区'] == ward) &
            (df['部屋サイズ_m2'].between(25, 35))
        ]
        if len(similar_conditions) > 0:
            avg_price = similar_conditions['家賃_円'].mean()
            print(f"{ward}: ¥{avg_price:,.0f} (近似条件)")

print("\nデータ生成完了! tokyo_rent_data_v2.csvファイルを使用してください。")